In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC50_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 0
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC50(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC50(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC50(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料


D:\Hu_code\ResNet18_ESC50\ESC50_kFold_1s\..\TrainDataPreprocessing.py:57: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
Data Loading Time: 129.76 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []


#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ", fold)
    print("----------------------------------------------------------------------------------------------------------------")

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=50)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    #------------------------------------------------
    #需要調整的地方
    #------------------------------------------------
    #學習率
    lr= 0.0032
    #epoch
    epoch= 90
    #訓練資料(要用幾秒訓練) data_1S/data_3S/data_5S
    trainData = data_1S
    #------------------------------------------------
    
    #紀錄訓練開始時間
    start_time = time.time()

    #訓練模型
    model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold,
                                                                setLR=lr,
                                                                setEpoch=epoch)
    
    #測試準確度
    test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
    test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
    test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

    #紀錄訓練結束時間
    end_time = time.time()

    #計算訓練耗時
    training_time = end_time - start_time

    #印出訓練耗時
    print("Training Time: {:.2f} seconds".format(training_time))

    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("訓練過程\n")
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    for i in range(len(train_loss)):
        # 寫入資料
        f.write("Fold: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
    f.write("Training Time: {:.2f} seconds\n".format(training_time))
    f.write("---------------------------------------------------------------------------------------------------------------\n")

    #--------------------------------------------------------------------

    
    kFoldTrainTime.append(training_time)
    kFoldTrainAccuracy.append(train_accuracy[-1])
    kFoldTestAccuracy1.append(test_accuracy1)
    kFoldTestAccuracy3.append(test_accuracy3)
    kFoldTestAccuracy5.append(test_accuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

----------------------------------------------------------------------------------------------------------------
fold:  1
----------------------------------------------------------------------------------------------------------------
可用的 GPU 数量: 1
Train Epoch: 1/90 Train_Loss: 2.818002462387085 Train_acc: 13.075001%
Train Epoch: 2/90 Train_Loss: 1.8208357095718384 Train_acc: 32.075001%
Train Epoch: 3/90 Train_Loss: 1.8649601936340332 Train_acc: 44.762501%
Train Epoch: 4/90 Train_Loss: 1.3685778379440308 Train_acc: 55.637501%
Train Epoch: 5/90 Train_Loss: 1.2476998567581177 Train_acc: 64.025002%
Train Epoch: 6/90 Train_Loss: 1.0012986660003662 Train_acc: 69.312500%
Train Epoch: 7/90 Train_Loss: 0.9809095859527588 Train_acc: 75.212502%
Train Epoch: 8/90 Train_Loss: 0.8503445386886597 Train_acc: 78.262505%
Train Epoch: 9/90 Train_Loss: 0.715117871761322 Train_acc: 80.887505%
Train Epoch: 10/90 Train_Loss: 0.5573292970657349 Train_acc: 83.875008%
Train Epoch: 11/90 Train_Loss: 0.622463822

Train Epoch: 16/90 Train_Loss: 0.29598188400268555 Train_acc: 91.550003%
Train Epoch: 17/90 Train_Loss: 0.2661401033401489 Train_acc: 92.250008%
Train Epoch: 18/90 Train_Loss: 0.11012345552444458 Train_acc: 92.400002%
Train Epoch: 19/90 Train_Loss: 0.37819817662239075 Train_acc: 92.612503%
Train Epoch: 20/90 Train_Loss: 0.3704932630062103 Train_acc: 92.662506%
Train Epoch: 21/90 Train_Loss: 0.3335120975971222 Train_acc: 92.650002%
Train Epoch: 22/90 Train_Loss: 0.19973735511302948 Train_acc: 92.775002%
Train Epoch: 23/90 Train_Loss: 0.35316547751426697 Train_acc: 92.612503%
Train Epoch: 24/90 Train_Loss: 0.22020407021045685 Train_acc: 92.825005%
Train Epoch: 25/90 Train_Loss: 0.11109275370836258 Train_acc: 92.825005%
Train Epoch: 26/90 Train_Loss: 0.2927682399749756 Train_acc: 92.800003%
Train Epoch: 27/90 Train_Loss: 0.23629595339298248 Train_acc: 92.400002%
Train Epoch: 28/90 Train_Loss: 0.16769322752952576 Train_acc: 91.337502%
Train Epoch: 29/90 Train_Loss: 0.4458388388156891 Train

Train Epoch: 34/90 Train_Loss: 0.25854790210723877 Train_acc: 93.000008%
Train Epoch: 35/90 Train_Loss: 0.13844464719295502 Train_acc: 93.000008%
Train Epoch: 36/90 Train_Loss: 0.17689989507198334 Train_acc: 92.975006%
Train Epoch: 37/90 Train_Loss: 0.08435647934675217 Train_acc: 92.950005%
Train Epoch: 38/90 Train_Loss: 0.4083375632762909 Train_acc: 93.087502%
Train Epoch: 39/90 Train_Loss: 0.29292574524879456 Train_acc: 93.025002%
Train Epoch: 40/90 Train_Loss: 0.07962267100811005 Train_acc: 93.037506%
Train Epoch: 41/90 Train_Loss: 0.22034716606140137 Train_acc: 93.125008%
Train Epoch: 42/90 Train_Loss: 0.4804982841014862 Train_acc: 93.162506%
Train Epoch: 43/90 Train_Loss: 0.1564219743013382 Train_acc: 93.137505%
Train Epoch: 44/90 Train_Loss: 0.3526075482368469 Train_acc: 93.050003%
Train Epoch: 45/90 Train_Loss: 0.1312359869480133 Train_acc: 93.162506%
Train Epoch: 46/90 Train_Loss: 0.14302144944667816 Train_acc: 93.175003%
Train Epoch: 47/90 Train_Loss: 0.23851622641086578 Train

Train Epoch: 52/90 Train_Loss: 0.2716589868068695 Train_acc: 92.787506%
Train Epoch: 53/90 Train_Loss: 0.08578424900770187 Train_acc: 92.762505%
Train Epoch: 54/90 Train_Loss: 0.29524528980255127 Train_acc: 92.775002%
Train Epoch: 55/90 Train_Loss: 0.1534019410610199 Train_acc: 92.775002%
Train Epoch: 56/90 Train_Loss: 0.23276126384735107 Train_acc: 92.737503%
Train Epoch: 57/90 Train_Loss: 0.1594645231962204 Train_acc: 92.762505%
Train Epoch: 58/90 Train_Loss: 0.0710614025592804 Train_acc: 92.775002%
Train Epoch: 59/90 Train_Loss: 0.13965347409248352 Train_acc: 92.800003%
Train Epoch: 60/90 Train_Loss: 0.45014792680740356 Train_acc: 92.787506%
Train Epoch: 61/90 Train_Loss: 0.3984489440917969 Train_acc: 92.787506%
Train Epoch: 62/90 Train_Loss: 0.3212331235408783 Train_acc: 92.800003%
Train Epoch: 63/90 Train_Loss: 0.1540960967540741 Train_acc: 92.787506%
Train Epoch: 64/90 Train_Loss: 0.13453498482704163 Train_acc: 92.775002%
Train Epoch: 65/90 Train_Loss: 0.2546384334564209 Train_ac

Train Epoch: 70/90 Train_Loss: 0.2452278882265091 Train_acc: 92.875008%
Train Epoch: 71/90 Train_Loss: 0.1612778902053833 Train_acc: 92.862503%
Train Epoch: 72/90 Train_Loss: 0.4021194577217102 Train_acc: 92.862503%
Train Epoch: 73/90 Train_Loss: 0.4448413550853729 Train_acc: 92.862503%
Train Epoch: 74/90 Train_Loss: 0.2285410463809967 Train_acc: 92.875008%
Train Epoch: 75/90 Train_Loss: 0.3101155757904053 Train_acc: 92.862503%
Train Epoch: 76/90 Train_Loss: 0.29540109634399414 Train_acc: 92.875008%
Train Epoch: 77/90 Train_Loss: 0.20170187950134277 Train_acc: 92.875008%
Train Epoch: 78/90 Train_Loss: 0.28890612721443176 Train_acc: 92.875008%
Train Epoch: 79/90 Train_Loss: 0.16809260845184326 Train_acc: 92.875008%
Train Epoch: 80/90 Train_Loss: 0.31118667125701904 Train_acc: 92.875008%
Train Epoch: 81/90 Train_Loss: 0.05100061744451523 Train_acc: 92.875008%
Train Epoch: 82/90 Train_Loss: 0.3450785279273987 Train_acc: 92.862503%
Train Epoch: 83/90 Train_Loss: 0.3043568730354309 Train_ac

In [5]:
#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    # 寫入資料
    f.write("Fold: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, kFoldTrainAccuracy[fold], kFoldTestAccuracy1[fold], kFoldTestAccuracy3[fold], kFoldTestAccuracy5[fold], kFoldTrainTime[fold]))
    f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(max(kFoldTrainAccuracy), max(kFoldTestAccuracy1), max(kFoldTestAccuracy3), max(kFoldTestAccuracy5)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(min(kFoldTrainAccuracy), min(kFoldTestAccuracy1), min(kFoldTestAccuracy3), min(kFoldTestAccuracy5)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(np.mean(kFoldTrainAccuracy), np.mean(kFoldTestAccuracy1), np.mean(kFoldTestAccuracy3), np.mean(kFoldTestAccuracy5)))

f.close()